In [1]:
import json
import pandas as pd
import os

In [2]:
def readJson(filename): 
    
    ##打开json文件,选择字段
    fp = open(filename,encoding='UTF-8')
    JsonObject = json.loads(fp.read())
    page = JsonObject['response']['list']
    
    ##读出用户昵称, 用户发文时间,发文内容
    for post in page:
        data = [post['trackback_author_nick'],post['trackback_author_name'],post['trackback_date'],post['content']]        
        df.loc[df.shape[0]]=data
    fp.close()

In [3]:
df = pd.DataFrame(columns=['nick','name','date','content'])

In [4]:
path = './附件/附件2-JPalljson/'
for fpathe,dirs,fs in os.walk(path):
    for f in fs:
        readJson(os.path.join(fpathe,f))
    print(df)
    


JSONDecodeError: Expecting value: line 1 column 1 (char 0)